### step 4

In [1]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from collections import Counter
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
import glob
from sklearn.semi_supervised import *
import pickle
from keras.applications import *
from keras.preprocessing.image import *
from keras.losses import mse, binary_crossentropy
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import random,math
from tqdm import tqdm ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle
from keras.regularizers import l2
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping

import resnet
from random_eraser import get_random_eraser
from mixup_generator import MixupGenerator

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

def getTrainData():
    X = []
    y = []

    for i in range(num_fold):
        fileX = os.path.join(base_data_path, 'X/X' + str(i+1) + '.npy')
        fileY = os.path.join(base_data_path, 'y/y' + str(i+1) + '.npy')
        
        X.append(np.load(fileX))
        y.append(np.load(fileY))

    X = np.array(X)
    y = np.array(y)

    return X, y

def split_data(X, y, idx):
    X_train = []
    y_train = []
    
    for i in range(num_fold):
        if i == idx:
            X_val = X[i]
            y_val = y[i]
            continue
        if X_train == []:
            X_train = X[i]
            y_train = y[i]
        else:
            X_train = np.concatenate((X_train, X[i]))
            y_train = np.concatenate((y_train, y[i]))

    return X_train, y_train, X_val, y_val

def normalize(X_train, X_val):
    X_train = (X_train - mean)/(std)
#     X_train = (X_train - min_)/range_
    X_val = (X_val - mean)/(std)
#     X_val = (X_val - min_)/range_

    return X_train, X_val


In [3]:

# min_ = np.swapaxes(min_,0,1)
# mean = np.swapaxes(mean,0,1)
# range_ = np.swapaxes(range_,0,1)
# std = np.swapaxes(std,0,1)

In [4]:
def train_valid(X_train,Y_train,X_valid,Y_valid,fold):
    model = [resnet.ResnetBuilder.build_resnet_18((1, X_train.shape[1], X_train.shape[2]), 41),
             resnet.ResnetBuilder.build_resnet_34((1,X_train.shape[1], X_train.shape[2]), 41),
             resnet.ResnetBuilder.build_resnet_50((1, X_train.shape[1], X_train.shape[2]), 41) ,]
#              resnet.ResnetBuilder.build_resnet_101((1, X_train.shape[1], X_train.shape[2]), 41),]
#              resnet.ResnetBuilder.build_resnet_152((1, X_train.shape[1], X_train.shape[2]), 41)]
    kk = random.randint(0, 2)#4)
    model = model[kk]
    print('using resnet model: '+str(kk))

    model.summary()
    if kk>=3:
        batchSize=[32]
    elif kk>2:
        batchSize=[32,64]#,128,256]
    elif kk>=1:
        batchSize=[32,64,128]
    else:
        batchSize=[32,64,128,256]
    batchSize = random.choice(batchSize)
    patien=100
    epoch=3000
    saveD = 'model/'+feature_type+'/'
    if not os.path.exists(saveD):
        os.makedirs(saveD)
    opt = Adam()#Nadam() #Adam(lr=2e-3,decay=1e-20)
    
    datagen = ImageDataGenerator(
#         featurewise_center=True,  # set input mean to 0 over the dataset
        width_shift_range=0.1*random.random(),
        height_shift_range=0.1*random.random(),
        shear_range=0.05*random.random(),
        preprocessing_function=get_random_eraser(p=0.82+0.13*random.random(),v_l=np.min(X_train), v_h=np.max(X_train)) # Trainset's boundaries.
    )
#     datagen.fit(X_train)
    generator = MixupGenerator(X_train, Y_train, alpha=1, 
                               batch_size=batchSize, datagen=datagen)

    model.compile(loss=['categorical_crossentropy'],optimizer=opt, metrics=['acc']) 
    logD = './logs/'+feature_type+'/'
    print('using resnet model: '+str(kk))
    if not os.path.exists(logD):
        os.makedirs(logD)
    history = History()
    callback=[
    #     ReduceLROnPlateau(monitor='loss', factor=0.5, patience=int(patien/2),
    #                                   min_lr=1e-4,mode='min', cooldown=1 ),
        EarlyStopping(patience=patien,monitor='val_loss',verbose=1,
                      mode='min'),
        ModelCheckpoint(saveD+'LGD_fold'+str(fold)+'_resnet'+str(kk)+'-.h5',
                        monitor='val_acc',verbose=1,save_best_only=True, 
                        save_weights_only=False,mode='max'),
        TensorBoard(log_dir=logD+'LGD_fold'+str(fold)+'_resnet'+str(kk)),
        history#,batch_size=batch_size, write_graph=True, write_grads=False, write_images=True)
    ]

    model.fit_generator(generator(),
                        steps_per_epoch=X_train.shape[0] // batchSize,
                        shuffle=True,
                        callbacks=callback, 
                        class_weight='auto',
                        validation_data=(X_valid,Y_valid),
                        max_queue_size = 32,
                        workers = 11,
#                         use_multiprocessing = True,
#                         batch_size=batchSize,
                        epochs=epoch
                       )
    return model,kk

In [5]:
def get_semi_data(X_train,Y_train):
    X_un_ver = np.load('feature/'+feature_type+'/semi/fbank4/X_un_ver.npy')
    X_test_ver = np.load('feature/'+feature_type+'/semi/fbank4/X_test_ver.npy')
    X = np.concatenate((X_un_ver,X_test_ver))
    Y_un_ver = np.load('feature/'+feature_type+'/semi/fbank4/Y_un_ver.npy')
    Y_test_ver = np.load('feature/'+feature_type+'/semi/fbank4/Y_test_ver.npy')
    Y = np.concatenate((Y_un_ver,Y_test_ver))
    Y = to_categorical(Y,num_classes=41)
    X_semi = np.concatenate((X_train,X))
    Y_semi = np.concatenate((Y_train,Y))
    X_semi , Y_semi = _shuffle(X_semi,Y_semi)
    print(X_semi.shape , Y_semi.shape)
    return X_semi , Y_semi

In [6]:
# model = resnet.ResnetBuilder.build_resnet_50((1, 64, 431), 41)
# model.summary()
def train_unverified(model,X_semi,Y_semi,fold,kk):
    name = glob.glob('model/'+feature_type+'/'+'LGD_fold'+str(fold)+'_resnet'+str(kk)+'-**')[0]
    print('semi loading: '+ name)
    model = load_model(name)
    if kk==4:
        batchSize=[32]
    elif kk>=2:
        batchSize=[32,64]#,128,256]
    elif kk>=1:
        batchSize=[32,64,128]
    else:
        batchSize=[32,64,128,256]
#     batchSize=[32,64,128,256] ##ERR?
    batchSize = random.choice(batchSize)
    patien=100
    epoch=3000
    saveD = 'model/'+feature_type+'/'
    opt = Adam(lr=0.0001,decay=1e-6)#Nadam() #Adam(lr=2e-3,decay=1e-20)
    
    
    datagen = ImageDataGenerator(
#         featurewise_center=True,  # set input mean to 0 over the dataset
#         featurewise_std_normalization=True,
        width_shift_range=0.05+0.15*random.random(),
        height_shift_range=0.05+0.15*random.random(),
        shear_range=0.084375+0.253125*random.random(),
        preprocessing_function=get_random_eraser(v_l=np.min(X_semi), v_h=np.max(X_semi)) # Trainset's boundaries.
    )
#     datagen.fit(X_semi)
    test_datagen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True)
    generator = MixupGenerator(X_semi, Y_semi, alpha=0.4+0.6*random.random(), 
                               batch_size=batchSize, datagen=datagen)
    

    model.compile(loss=['categorical_crossentropy'],optimizer=opt, metrics=['acc']) 
    logD = './logs/'+feature_type+'/'
    history = History()
    callback=[
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=int(patien/10),min_lr=4e-6,
                          mode='min', cooldown=1,verbose=1 ), #0.2,/25
        EarlyStopping(patience=patien,monitor='val_loss',verbose=1,
                      mode='min'),
        ModelCheckpoint(saveD+'LGD_fold'+str(fold)+'_semi_resnet'+str(kk)+'.h5',
                        monitor='val_acc',verbose=1,save_best_only=True, 
                        save_weights_only=False,
                        mode='max'),
        TensorBoard(log_dir=logD+'LGD_fold'+str(fold)+'_semi_resnet'+str(kk)),
        history
    ]
    model.fit_generator(generator(),
                        steps_per_epoch=X_semi.shape[0] // batchSize,
                        shuffle=True,
                        callbacks=callback, 
                        class_weight='auto',
                        validation_data=(X_valid,Y_valid),
                        max_queue_size = 32,
                        workers = 11,
#                         use_multiprocessing = True,
#                         batch_size=batchSize,
                        epochs=epoch,
                        initial_epoch = int(patien/25)
                       )
#     model.fit(X_semi,Y_semi,
#               shuffle=True,
#               callbacks=callback, 
#               class_weight='auto',
#               validation_data=(X_valid,Y_valid),
#               batch_size=batchSize,
#               epochs=epoch)
    return model

In [7]:
feature_type = 'mfcc7'

In [8]:
# normalized parameters
mean = np.load('feature/'+feature_type+'/mean.npy')
std = np.load('feature/'+feature_type+'/std.npy')
min_ = np.load('feature/'+feature_type+'/min.npy')
range_ = np.load('feature/'+feature_type+'/range.npy')


base_path = 'feature/'+feature_type+'/'#'/tmp2/b03902110/newphase1'
base_data_path = 'feature/'+feature_type+'/'#os.path.join(base_path, 'data')
num_fold = 10

val_set_num = [7,8,9]#str(sys.argv[1])

In [9]:
# fold=0
for fold in val_set_num:
    X, y = getTrainData()
    # X = np.swapaxes(X,2,3)
    X_train, Y_train, X_valid, Y_valid = split_data(X, y, fold) #fold
    # X_train, X_valid = normalize(X_train, X_valid)
    print(X_train.shape, Y_train.shape)

    # X_train = np.swapaxes(X_train,1,3)
    # X_valid = np.swapaxes(X_valid,1,3)
    print("===train verified_fold"+str(fold)+'_'+feature_type+'===')
    model,model_num = train_valid(X_train,Y_train,X_valid,Y_valid,fold)
    X_semi , Y_semi = get_semi_data(X_train,Y_train)
    print('===train semi_'+str(fold)+'===')
    model_semi = train_unverified(model,X_semi,Y_semi,fold,model_num)


/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


(3339, 60, 259, 1) (3339, 41)
===train verified_fold6_mfcc7===
using resnet model: 2
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 60, 259, 1)   0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 30, 130, 64)  3200        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 30, 130, 64)  256         conv2d_57[0][0]                  
__________________________________________________________________________________________________
activation_51 (Activation)      (None, 30, 130, 64)  0           batch_normalization_53[0][0]     
________________________

Epoch 1/3000
52/52 [==============================] - 16s 302ms/step - loss: 9.0078 - acc: 0.0604 - val_loss: 8.8012 - val_acc: 0.0647

Epoch 00001: val_acc improved from -inf to 0.06469, saving model to model/mfcc7/LGD_fold6_resnet2-.h5
Epoch 2/3000
52/52 [==============================] - 9s 165ms/step - loss: 7.7923 - acc: 0.0739 - val_loss: 7.1946 - val_acc: 0.0997

Epoch 00002: val_acc improved from 0.06469 to 0.09973, saving model to model/mfcc7/LGD_fold6_resnet2-.h5
Epoch 3/3000
52/52 [==============================] - 9s 165ms/step - loss: 6.8076 - acc: 0.0995 - val_loss: 6.8510 - val_acc: 0.0782

Epoch 00003: val_acc did not improve from 0.09973
Epoch 4/3000
52/52 [==============================] - 9s 166ms/step - loss: 6.0597 - acc: 0.1187 - val_loss: 6.1507 - val_acc: 0.0485

Epoch 00004: val_acc did not improve from 0.09973
Epoch 5/3000
52/52 [==============================] - 9s 166ms/step - loss: 5.5071 - acc: 0.1385 - val_loss: 5.0723 - val_acc: 0.1725

Epoch 00005: val_

52/52 [==============================] - 9s 167ms/step - loss: 2.7563 - acc: 0.4567 - val_loss: 2.3626 - val_acc: 0.4286

Epoch 00042: val_acc did not improve from 0.48248
Epoch 43/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.7926 - acc: 0.4555 - val_loss: 2.2227 - val_acc: 0.4394

Epoch 00043: val_acc did not improve from 0.48248
Epoch 44/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.7858 - acc: 0.4444 - val_loss: 2.5051 - val_acc: 0.3747

Epoch 00044: val_acc did not improve from 0.48248
Epoch 45/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.7206 - acc: 0.4627 - val_loss: 2.0551 - val_acc: 0.5229

Epoch 00045: val_acc improved from 0.48248 to 0.52291, saving model to model/mfcc7/LGD_fold6_resnet2-.h5
Epoch 46/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.7352 - acc: 0.4769 - val_loss: 2.2166 - val_acc: 0.5013

Epoch 00046: val_acc did not improve from 0.52291
Epoch 47/3000
52/52 [========

52/52 [==============================] - 9s 167ms/step - loss: 2.4187 - acc: 0.5691 - val_loss: 1.6502 - val_acc: 0.6334

Epoch 00084: val_acc improved from 0.61995 to 0.63342, saving model to model/mfcc7/LGD_fold6_resnet2-.h5
Epoch 85/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.4170 - acc: 0.5673 - val_loss: 1.7660 - val_acc: 0.5633

Epoch 00085: val_acc did not improve from 0.63342
Epoch 86/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.4438 - acc: 0.5634 - val_loss: 1.8612 - val_acc: 0.5580

Epoch 00086: val_acc did not improve from 0.63342
Epoch 87/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.4751 - acc: 0.5493 - val_loss: 2.3727 - val_acc: 0.4232

Epoch 00087: val_acc did not improve from 0.63342
Epoch 88/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.4273 - acc: 0.5628 - val_loss: 1.9449 - val_acc: 0.5391

Epoch 00088: val_acc did not improve from 0.63342
Epoch 89/3000
52/52 [========

...

53/53 [==============================] - 6s 117ms/step - loss: 1.3221 - acc: 0.8650 - val_loss: 0.7326 - val_acc: 0.9003

Epoch 00308: val_acc did not improve from 0.90566
Epoch 309/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2890 - acc: 0.8806 - val_loss: 0.7327 - val_acc: 0.9003

Epoch 00309: val_acc did not improve from 0.90566
Epoch 310/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2897 - acc: 0.8659 - val_loss: 0.7328 - val_acc: 0.9003

Epoch 00310: val_acc did not improve from 0.90566
Epoch 311/3000
53/53 [==============================] - 6s 118ms/step - loss: 1.3001 - acc: 0.8656 - val_loss: 0.7332 - val_acc: 0.9003

Epoch 00311: val_acc did not improve from 0.90566
Epoch 312/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.3035 - acc: 0.8623 - val_loss: 0.7352 - val_acc: 0.9003

Epoch 00312: val_acc did not improve from 0.90566
Epoch 313/3000
53/53 [==============================] - 6s 118ms/step - loss: 1.

53/53 [==============================] - 6s 117ms/step - loss: 1.2935 - acc: 0.8735 - val_loss: 0.7304 - val_acc: 0.8949

Epoch 00352: val_acc did not improve from 0.90566
Epoch 353/3000
53/53 [==============================] - 6s 118ms/step - loss: 1.2987 - acc: 0.8691 - val_loss: 0.7304 - val_acc: 0.8976

Epoch 00353: val_acc did not improve from 0.90566
Epoch 354/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2936 - acc: 0.8617 - val_loss: 0.7296 - val_acc: 0.8976

Epoch 00354: val_acc did not improve from 0.90566
Epoch 355/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2881 - acc: 0.8620 - val_loss: 0.7292 - val_acc: 0.8976

Epoch 00355: val_acc did not improve from 0.90566
Epoch 356/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2885 - acc: 0.8729 - val_loss: 0.7275 - val_acc: 0.8976

Epoch 00356: val_acc did not improve from 0.90566
Epoch 357/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.

53/53 [==============================] - 6s 117ms/step - loss: 1.2876 - acc: 0.8665 - val_loss: 0.7308 - val_acc: 0.9003

Epoch 00396: val_acc did not improve from 0.90566
Epoch 397/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2779 - acc: 0.8700 - val_loss: 0.7289 - val_acc: 0.8976

Epoch 00397: val_acc did not improve from 0.90566
Epoch 398/3000
53/53 [==============================] - 6s 118ms/step - loss: 1.2639 - acc: 0.8747 - val_loss: 0.7298 - val_acc: 0.8976

Epoch 00398: val_acc did not improve from 0.90566
Epoch 399/3000
53/53 [==============================] - 6s 118ms/step - loss: 1.2938 - acc: 0.8756 - val_loss: 0.7315 - val_acc: 0.8976

Epoch 00399: val_acc did not improve from 0.90566
Epoch 400/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2798 - acc: 0.8721 - val_loss: 0.7308 - val_acc: 0.8976

Epoch 00400: val_acc did not improve from 0.90566
Epoch 401/3000
53/53 [==============================] - 6s 118ms/step - loss: 1.

53/53 [==============================] - 6s 117ms/step - loss: 1.2708 - acc: 0.8768 - val_loss: 0.7326 - val_acc: 0.9003

Epoch 00440: val_acc did not improve from 0.90566
Epoch 441/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2832 - acc: 0.8726 - val_loss: 0.7316 - val_acc: 0.9003

Epoch 00441: val_acc did not improve from 0.90566
Epoch 442/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2966 - acc: 0.8715 - val_loss: 0.7305 - val_acc: 0.9003

Epoch 00442: val_acc did not improve from 0.90566
Epoch 443/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2741 - acc: 0.8726 - val_loss: 0.7311 - val_acc: 0.9003

Epoch 00443: val_acc did not improve from 0.90566
Epoch 444/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2872 - acc: 0.8785 - val_loss: 0.7321 - val_acc: 0.9003

Epoch 00444: val_acc did not improve from 0.90566
Epoch 445/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.

53/53 [==============================] - 6s 117ms/step - loss: 1.2837 - acc: 0.8638 - val_loss: 0.7295 - val_acc: 0.9030

Epoch 00484: val_acc did not improve from 0.90566
Epoch 00484: early stopping


In [ ]:
#MFCC7
#0=>0.84367(resnet3_semi)/0.83288(not semi)
#1=>0.84367(resnet3_semi)/0.81132(not semi)
#2=>0.84097(resnet2_semi)/0.81671(not semi)
#3=>0.88410(resnet1_semi)/0.84367
#4=>0.82210(resnet4_semi)/0.74663(not semi)
#5 => 0.82749(resnet4_semi)/0.77628
#6 => 0.85445(resnet2_semi)/0.78437
#7 => 0.82749(resnet3_semi)/0.76280
#8 => 0.83288(resnet3_semi)/0.78706
# 9=> 0.90566(resnet1_semi)/0.87332

In [10]:
#MFCC6
# 0=> 0.81941 (not semi)
# 1=>0.83019 (semi)
# 2=>0.81941 (semi)
# 3=>0.85984 (resnet1_not semi)、0.78437
# 4=>0.84367 (renet1_not semi)、0.81132
# 5=> 0.85175(resnet3_semi)、0.82749(not semi)
#6 => 0.85904(resnet4_semi)、0.79784(not semi)
# 7=>0.88949(resnet2_semi)、0.83558(not semi)
# 8=>0.83558(resnet1 not semi)
# 9=>0.86792(resnet2_semi)、0.8112(not semi)

In [13]:
# np.load('feature/mfcc6/fname_unverified.npy')

array(['00044347.wav', '002d256b.wav', '003b91e8.wav', ...,
       'fff37590.wav', 'fff44ac6.wav', 'fff6a13d.wav'], dtype=object)

### Step 8
# Co-Train

In [2]:
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

def getTrainData():
    X = []
    y = []

    for i in range(num_fold):
        fileX = os.path.join(base_data_path, 'X/X' + str(i+1) + '.npy')
        fileY = os.path.join(base_data_path, 'y/y' + str(i+1) + '.npy')
        
        X.append(np.load(fileX))
        y.append(np.load(fileY))

    X = np.array(X)
    y = np.array(y)

    return X, y

def split_data(X, y, idx):
    X_train = []
    y_train = []
    
    for i in range(num_fold):
        if i == idx:
            X_val = X[i]
            y_val = y[i]
            continue
        if X_train == []:
            X_train = X[i]
            y_train = y[i]
        else:
            X_train = np.concatenate((X_train, X[i]))
            y_train = np.concatenate((y_train, y[i]))

    return X_train, y_train, X_val, y_val

def normalize(X_train, X_val):
    X_train = (X_train - mean)/(std)
#     X_train = (X_train - min_)/range_
    X_val = (X_val - mean)/(std)
#     X_val = (X_val - min_)/range_

    return X_train, X_val


In [3]:
# model = resnet.ResnetBuilder.build_resnet_50((1, 64, 431), 41)
# model.summary()
def train_unverified(X_semi,Y_semi,fold):
    name = glob.glob('model/'+feature_type+'/'+'**_fold'+str(fold)+'_res**')[0]
    if fold in [0,1,2,3,4,5]: # MFCC6
        name = glob.glob('model/'+feature_type+'/'+'**_fold'+str(fold)+'_co_**')[0]
    print('semi loading: '+ name)
    model = load_model(name)
    model.summary()
    if 'resnet4'in name:
        batchSize=[32]
    elif 'resnet2' in name or 'resnet3' in name:
        batchSize=[32,64]#,128,256]
    elif 'resnet1' in name:
        batchSize=[32,64,128]
    else:
        batchSize=[32,64]
#     batchSize=[32,64,128,256] ##ERR?
    batchSize = random.choice(batchSize)
    patien=30
    epoch=3000
    saveD = 'model/'+feature_type+'/'
    opt = Adam(lr=0.0001,decay=1e-6)#Nadam() #Adam(lr=2e-3,decay=1e-20)
    
    
    datagen = ImageDataGenerator(
#         featurewise_center=True,  # set input mean to 0 over the dataset
#         featurewise_std_normalization=True,
        width_shift_range=0.05+0.3*random.random(),
        height_shift_range=0.05+0.3*random.random(),
        shear_range=0.084375+0.253125*random.random(),
        preprocessing_function=get_random_eraser(v_l=np.min(X_semi), v_h=np.max(X_semi)) # Trainset's boundaries.
    )
#     datagen.fit(X_semi)
    test_datagen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True)
    generator = MixupGenerator(X_semi, Y_semi, alpha=0.4+0.6*random.random(), 
                               batch_size=batchSize, datagen=datagen)
    

    model.compile(loss=['categorical_crossentropy'],optimizer=opt, metrics=['acc']) 
    logD = './logs/'+feature_type+'/'
    history = History()
    callback=[
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=int(patien/4),min_lr=4e-6,
                          mode='min', cooldown=1,verbose=1 ), #0.2,/25 #0.5/5/3e-6 #0.3/10/1e-6
        EarlyStopping(patience=patien,monitor='val_loss',verbose=1,
                      mode='min'),
        ModelCheckpoint(saveD+'LGD_fold'+str(fold)+'_co_resnet'+'.h5',
                        monitor='val_acc',verbose=1,save_best_only=True, 
                        save_weights_only=False,
                        mode='max'),
        TensorBoard(log_dir=logD+'LGD_fold'+str(fold)+'_co_resnet'),
        history
    ]
    model.fit_generator(generator(),
                        steps_per_epoch=2*X_semi.shape[0] // batchSize,
                        shuffle=True,
                        callbacks=callback, 
                        class_weight='auto',
                        validation_data=(X_valid,Y_valid),
                        max_queue_size = 32,
                        workers = 11,
#                         use_multiprocessing = True,
#                         batch_size=batchSize,
                        epochs=epoch,
#                         initial_epoch = int(patien/20)
                       )
#     model.fit(X_semi,Y_semi,
#               shuffle=True,
#               callbacks=callback, 
#               class_weight='auto',
#               validation_data=(X_valid,Y_valid),
#               batch_size=batchSize,
#               epochs=epoch)
    return model

In [4]:
def get_semi_data(X_train,Y_train):
    X = np.load('feature/'+feature_type+'/semi/'+who+'/X_co.npy')
#     X_test_ver = np.load('feature/'+feature_type+'/semi/'+who+'/X_test_ver.npy')
#     X = np.concatenate((X_un_ver,X_test_ver))
    Y = np.load('feature/'+feature_type+'/semi/'+who+'/Y_co.npy')
#     Y_test_ver = np.load('feature/'+feature_type+'/semi/'+who+'/Y_test_ver.npy')
#     Y = np.concatenate((Y_un_ver,Y_test_ver))
    Y = to_categorical(Y,num_classes=41)
    X_semi = np.concatenate((X_train,X))
    Y_semi = np.concatenate((Y_train,Y))
    X_semi , Y_semi = _shuffle(X_semi,Y_semi)
    print(X_semi.shape , Y_semi.shape)
    return X_semi , Y_semi

In [5]:
feature_type = 'mfcc6'
who = 'jerry_mfcc3_resnet34_mixup_cotrain_Y'

In [6]:
base_path = 'feature/'+feature_type+'/'#'/tmp2/b03902110/newphase1'
base_data_path = 'feature/'+feature_type+'/'#os.path.join(base_path, 'data')
num_fold = 10

val_set_num = [0,1,2,3,4,5]#str(sys.argv[1])

In [ ]:
for fold in val_set_num:
    X, y = getTrainData()
    # X = np.swapaxes(X,2,3)
    X_train, Y_train, X_valid, Y_valid = split_data(X, y, fold) #fold
    # X_train, X_valid = normalize(X_train, X_valid)
    print(X_train.shape, Y_train.shape)

    # X_train = np.swapaxes(X_train,1,3)
    # X_valid = np.swapaxes(X_valid,1,3)
#     print("===train verified_fold"+str(fold)+'_'+feature_type+'===')
#     model,model_num = train_valid(X_train,Y_train,X_valid,Y_valid,fold)
    X_semi , Y_semi = get_semi_data(X_train,Y_train)
    print('===train semi_'+str(fold)+'===')
    model_semi = train_unverified(X_semi,Y_semi,fold)

#MFCC7
#7=>
#8=>0.84906
#9=>0.90566

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


(3339, 60, 259, 1) (3339, 41)
(6360, 60, 259, 1) (6360, 41)
===train semi_9===
semi loading: model/mfcc7/LGD_fold9_co_resnet.h5
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 60, 259, 1)   0                                            
__________________________________________________________________________________________________
conv2d_660 (Conv2D)             (None, 30, 130, 64)  3200        input_14[0][0]                   
__________________________________________________________________________________________________
batch_normalization_628 (BatchN (None, 30, 130, 64)  256         conv2d_660[0][0]                 
__________________________________________________________________________________________________
activation_615 (Activation)     (None, 30, 130, 64)  0           batch_normaliza

Epoch 1/3000
198/198 [==============================] - 24s 121ms/step - loss: 1.4865 - acc: 0.8028 - val_loss: 0.6869 - val_acc: 0.8895

Epoch 00001: val_acc improved from -inf to 0.88949, saving model to model/mfcc7/LGD_fold9_co_resnet.h5
Epoch 2/3000
198/198 [==============================] - 19s 96ms/step - loss: 1.4630 - acc: 0.8086 - val_loss: 0.6689 - val_acc: 0.8922

Epoch 00002: val_acc improved from 0.88949 to 0.89218, saving model to model/mfcc7/LGD_fold9_co_resnet.h5
Epoch 3/3000
198/198 [==============================] - 19s 96ms/step - loss: 1.4465 - acc: 0.8163 - val_loss: 0.6682 - val_acc: 0.8976

Epoch 00003: val_acc improved from 0.89218 to 0.89757, saving model to model/mfcc7/LGD_fold9_co_resnet.h5
Epoch 4/3000
198/198 [==============================] - 19s 96ms/step - loss: 1.4410 - acc: 0.8194 - val_loss: 0.6729 - val_acc: 0.8976

Epoch 00004: val_acc did not improve from 0.89757
Epoch 5/3000
198/198 [==============================] - 19s 96ms/step - loss: 1.4474 -

198/198 [==============================] - 19s 97ms/step - loss: 1.3451 - acc: 0.8337 - val_loss: 0.6874 - val_acc: 0.8868

Epoch 00043: val_acc did not improve from 0.90566
Epoch 44/3000
198/198 [==============================] - 19s 97ms/step - loss: 1.3311 - acc: 0.8360 - val_loss: 0.6809 - val_acc: 0.8814

Epoch 00044: val_acc did not improve from 0.90566
Epoch 45/3000
198/198 [==============================] - 19s 97ms/step - loss: 1.3375 - acc: 0.8378 - val_loss: 0.6383 - val_acc: 0.8868

Epoch 00045: val_acc did not improve from 0.90566
Epoch 46/3000
198/198 [==============================] - 19s 97ms/step - loss: 1.3301 - acc: 0.8401 - val_loss: 0.6702 - val_acc: 0.8949

Epoch 00046: val_acc did not improve from 0.90566
Epoch 47/3000
198/198 [==============================] - 19s 97ms/step - loss: 1.3335 - acc: 0.8342 - val_loss: 0.6625 - val_acc: 0.8922

Epoch 00047: val_acc did not improve from 0.90566
Epoch 48/3000
198/198 [==============================] - 19s 97ms/step - l


Epoch 00086: val_acc did not improve from 0.90566
Epoch 87/3000
198/198 [==============================] - 19s 97ms/step - loss: 1.2578 - acc: 0.8461 - val_loss: 0.6619 - val_acc: 0.8787

Epoch 00087: val_acc did not improve from 0.90566
Epoch 88/3000
198/198 [==============================] - 19s 97ms/step - loss: 1.2562 - acc: 0.8461 - val_loss: 0.6546 - val_acc: 0.8814

Epoch 00088: val_acc did not improve from 0.90566
Epoch 89/3000
198/198 [==============================] - 19s 97ms/step - loss: 1.2591 - acc: 0.8446 - val_loss: 0.7028 - val_acc: 0.8787

Epoch 00089: val_acc did not improve from 0.90566
Epoch 90/3000
198/198 [==============================] - 19s 97ms/step - loss: 1.2661 - acc: 0.8484 - val_loss: 0.6662 - val_acc: 0.8868

Epoch 00090: val_acc did not improve from 0.90566
Epoch 91/3000
198/198 [==============================] - 19s 97ms/step - loss: 1.2506 - acc: 0.8494 - val_loss: 0.6743 - val_acc: 0.8787

Epoch 00091: val_acc did not improve from 0.90566
Epoch 92/

...

(3339, 60, 259, 1) (3339, 41)
(6360, 60, 259, 1) (6360, 41)
===train semi_8===
semi loading: model/mfcc7/LGD_semi_fold8_resnet3.h5
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 60, 259, 1)   0                                            
__________________________________________________________________________________________________
conv2d_536 (Conv2D)             (None, 30, 130, 64)  3200        input_12[0][0]                   
__________________________________________________________________________________________________
batch_normalization_509 (BatchN (None, 30, 130, 64)  256         conv2d_536[0][0]                 
__________________________________________________________________________________________________
activation_498 (Activation)     (None, 30, 130, 64)  0           batch_normal

Epoch 1/3000
198/198 [==============================] - 66s 332ms/step - loss: 2.2143 - acc: 0.5950 - val_loss: 0.9784 - val_acc: 0.8059

Epoch 00001: val_acc improved from -inf to 0.80593, saving model to model/mfcc7/LGD_fold8_co_resnet.h5
Epoch 2/3000
198/198 [==============================] - 53s 268ms/step - loss: 1.9740 - acc: 0.6676 - val_loss: 0.9554 - val_acc: 0.8086

Epoch 00002: val_acc improved from 0.80593 to 0.80863, saving model to model/mfcc7/LGD_fold8_co_resnet.h5
Epoch 3/3000
198/198 [==============================] - 53s 269ms/step - loss: 1.9026 - acc: 0.6884 - val_loss: 0.9152 - val_acc: 0.8113

Epoch 00003: val_acc improved from 0.80863 to 0.81132, saving model to model/mfcc7/LGD_fold8_co_resnet.h5
Epoch 4/3000
198/198 [==============================] - 53s 269ms/step - loss: 1.8588 - acc: 0.7074 - val_loss: 0.9790 - val_acc: 0.8005

Epoch 00004: val_acc did not improve from 0.81132
Epoch 5/3000
198/198 [==============================] - 53s 269ms/step - loss: 1.82

198/198 [==============================] - 53s 269ms/step - loss: 1.5836 - acc: 0.7792 - val_loss: 0.8920 - val_acc: 0.8302

Epoch 00042: val_acc did not improve from 0.84636
Epoch 43/3000
198/198 [==============================] - 53s 270ms/step - loss: 1.5714 - acc: 0.7914 - val_loss: 0.8664 - val_acc: 0.8248

Epoch 00043: val_acc did not improve from 0.84636
Epoch 44/3000
198/198 [==============================] - 53s 270ms/step - loss: 1.5759 - acc: 0.7877 - val_loss: 0.8836 - val_acc: 0.8221

Epoch 00044: val_acc did not improve from 0.84636
Epoch 45/3000
198/198 [==============================] - 53s 270ms/step - loss: 1.5641 - acc: 0.7863 - val_loss: 0.8466 - val_acc: 0.8275

Epoch 00045: val_acc did not improve from 0.84636
Epoch 46/3000
198/198 [==============================] - 53s 270ms/step - loss: 1.5638 - acc: 0.7884 - val_loss: 0.9197 - val_acc: 0.8356

Epoch 00046: val_acc did not improve from 0.84636
Epoch 47/3000
198/198 [==============================] - 53s 270ms/st

198/198 [==============================] - 53s 270ms/step - loss: 1.4902 - acc: 0.8097 - val_loss: 0.8888 - val_acc: 0.8302

Epoch 00085: val_acc did not improve from 0.84906
Epoch 86/3000
198/198 [==============================] - 53s 269ms/step - loss: 1.4790 - acc: 0.8070 - val_loss: 0.8783 - val_acc: 0.8329

Epoch 00086: val_acc did not improve from 0.84906
Epoch 87/3000
198/198 [==============================] - 53s 269ms/step - loss: 1.4743 - acc: 0.8073 - val_loss: 0.8607 - val_acc: 0.8248

Epoch 00087: val_acc did not improve from 0.84906
Epoch 88/3000
198/198 [==============================] - 53s 269ms/step - loss: 1.4993 - acc: 0.7976 - val_loss: 0.8515 - val_acc: 0.8275

Epoch 00088: val_acc did not improve from 0.84906
Epoch 89/3000
198/198 [==============================] - 53s 270ms/step - loss: 1.4739 - acc: 0.8107 - val_loss: 0.8437 - val_acc: 0.8302

Epoch 00089: val_acc did not improve from 0.84906
Epoch 90/3000
198/198 [==============================] - 53s 269ms/st

Epoch 1/3000
397/397 [==============================] - 83s 210ms/step - loss: 1.5498 - acc: 0.8025 - val_loss: 0.8631 - val_acc: 0.8167

Epoch 00001: val_acc improved from -inf to 0.81671, saving model to model/mfcc7/LGD_fold7_co_resnet.h5
Epoch 2/3000
397/397 [==============================] - 70s 177ms/step - loss: 1.5352 - acc: 0.8068 - val_loss: 0.8651 - val_acc: 0.8140

Epoch 00002: val_acc did not improve from 0.81671
Epoch 3/3000
397/397 [==============================] - 71s 178ms/step - loss: 1.5311 - acc: 0.8049 - val_loss: 0.8314 - val_acc: 0.8221

Epoch 00003: val_acc improved from 0.81671 to 0.82210, saving model to model/mfcc7/LGD_fold7_co_resnet.h5
Epoch 4/3000
397/397 [==============================] - 71s 178ms/step - loss: 1.5257 - acc: 0.8071 - val_loss: 0.8863 - val_acc: 0.8167

Epoch 00004: val_acc did not improve from 0.82210
Epoch 5/3000
397/397 [==============================] - 71s 178ms/step - loss: 1.5174 - acc: 0.8063 - val_loss: 0.8201 - val_acc: 0.8356

E

In [7]:
for fold in val_set_num:
    X, y = getTrainData()
    # X = np.swapaxes(X,2,3)
    X_train, Y_train, X_valid, Y_valid = split_data(X, y, fold) #fold
    # X_train, X_valid = normalize(X_train, X_valid)
    print(X_train.shape, Y_train.shape)

    # X_train = np.swapaxes(X_train,1,3)
    # X_valid = np.swapaxes(X_valid,1,3)
#     print("===train verified_fold"+str(fold)+'_'+feature_type+'===')
#     model,model_num = train_valid(X_train,Y_train,X_valid,Y_valid,fold)
    X_semi , Y_semi = get_semi_data(X_train,Y_train)
    print('===train semi_'+str(fold)+'===')
    model_semi = train_unverified(X_semi,Y_semi,fold)
# MFCC6



/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


(3339, 64, 431, 1) (3339, 41)
(6250, 64, 431, 1) (6250, 41)
===train semi_9===
semi loading: model/mfcc6/LGD_semi_fold9_resnet2.h5
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 64, 431, 1)   0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 32, 216, 64)  3200        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 216, 64)  256         conv2d_57[0][0]                  
__________________________________________________________________________________________________
activation_51 (Activation)      (None, 32, 216, 64)  0           batch_normal

Epoch 1/3000
390/390 [==============================] - 60s 154ms/step - loss: 1.7796 - acc: 0.7538 - val_loss: 0.9671 - val_acc: 0.8437

Epoch 00001: val_acc improved from -inf to 0.84367, saving model to model/mfcc6/LGD_fold9_co_resnet.h5
Epoch 2/3000
390/390 [==============================] - 53s 136ms/step - loss: 1.6576 - acc: 0.7913 - val_loss: 0.9901 - val_acc: 0.8437

Epoch 00002: val_acc did not improve from 0.84367
Epoch 3/3000
390/390 [==============================] - 53s 136ms/step - loss: 1.6115 - acc: 0.7986 - val_loss: 0.9155 - val_acc: 0.8544

Epoch 00003: val_acc improved from 0.84367 to 0.85445, saving model to model/mfcc6/LGD_fold9_co_resnet.h5
Epoch 4/3000
390/390 [==============================] - 53s 136ms/step - loss: 1.5790 - acc: 0.8039 - val_loss: 1.0235 - val_acc: 0.8383

Epoch 00004: val_acc did not improve from 0.85445
Epoch 5/3000
390/390 [==============================] - 53s 136ms/step - loss: 1.5633 - acc: 0.8026 - val_loss: 0.9366 - val_acc: 0.8518

E


Epoch 00042: val_acc did not improve from 0.87062
Epoch 43/3000
390/390 [==============================] - 53s 136ms/step - loss: 1.2969 - acc: 0.8453 - val_loss: 0.8839 - val_acc: 0.8518

Epoch 00043: val_acc did not improve from 0.87062
Epoch 44/3000
390/390 [==============================] - 53s 136ms/step - loss: 1.2952 - acc: 0.8416 - val_loss: 0.9055 - val_acc: 0.8248

Epoch 00044: val_acc did not improve from 0.87062
Epoch 45/3000
390/390 [==============================] - 53s 136ms/step - loss: 1.2802 - acc: 0.8503 - val_loss: 0.8181 - val_acc: 0.8491

Epoch 00045: val_acc did not improve from 0.87062
Epoch 46/3000
390/390 [==============================] - 53s 136ms/step - loss: 1.2922 - acc: 0.8463 - val_loss: 0.8916 - val_acc: 0.8329

Epoch 00046: val_acc did not improve from 0.87062
Epoch 47/3000
390/390 [==============================] - 53s 136ms/step - loss: 1.2750 - acc: 0.8454 - val_loss: 0.8337 - val_acc: 0.8464

Epoch 00047: val_acc did not improve from 0.87062
Epoc

390/390 [==============================] - 53s 136ms/step - loss: 1.2006 - acc: 0.8604 - val_loss: 0.8219 - val_acc: 0.8329

Epoch 00086: val_acc did not improve from 0.87332
Epoch 87/3000
390/390 [==============================] - 53s 136ms/step - loss: 1.2013 - acc: 0.8579 - val_loss: 0.8531 - val_acc: 0.8356

Epoch 00087: val_acc did not improve from 0.87332
Epoch 88/3000
390/390 [==============================] - 53s 136ms/step - loss: 1.2078 - acc: 0.8529 - val_loss: 0.8652 - val_acc: 0.8356

Epoch 00088: val_acc did not improve from 0.87332
Epoch 00088: early stopping


...

Epoch 1/3000
390/390 [==============================] - 42s 108ms/step - loss: 2.0031 - acc: 0.7454 - val_loss: 0.9261 - val_acc: 0.7978

Epoch 00001: val_acc improved from -inf to 0.79784, saving model to model/mfcc6/LGD_fold3_co_resnet.h5
Epoch 2/3000
390/390 [==============================] - 37s 94ms/step - loss: 2.1345 - acc: 0.7183 - val_loss: 5.5527 - val_acc: 0.5148

Epoch 00002: val_acc did not improve from 0.79784
Epoch 3/3000
390/390 [==============================] - 37s 94ms/step - loss: 2.0774 - acc: 0.6997 - val_loss: 1.1356 - val_acc: 0.7520

Epoch 00003: val_acc did not improve from 0.79784
Epoch 4/3000
390/390 [==============================] - 37s 94ms/step - loss: 1.7720 - acc: 0.7300 - val_loss: 1.0821 - val_acc: 0.7547

Epoch 00004: val_acc did not improve from 0.79784
Epoch 5/3000
390/390 [==============================] - 37s 94ms/step - loss: 1.7905 - acc: 0.7333 - val_loss: 1.3461 - val_acc: 0.7089

Epoch 00005: val_acc did not improve from 0.79784
Epoch 6/300

390/390 [==============================] - 37s 94ms/step - loss: 1.4871 - acc: 0.8073 - val_loss: 0.8813 - val_acc: 0.8113

Epoch 00042: val_acc did not improve from 0.83019
Epoch 43/3000
390/390 [==============================] - 37s 94ms/step - loss: 1.4715 - acc: 0.8088 - val_loss: 0.8737 - val_acc: 0.8221

Epoch 00043: val_acc did not improve from 0.83019
Epoch 44/3000
390/390 [==============================] - 37s 94ms/step - loss: 1.4627 - acc: 0.8171 - val_loss: 0.8846 - val_acc: 0.8221

Epoch 00044: val_acc did not improve from 0.83019
Epoch 45/3000
390/390 [==============================] - 37s 94ms/step - loss: 1.4705 - acc: 0.8126 - val_loss: 0.8670 - val_acc: 0.8221

Epoch 00045: val_acc did not improve from 0.83019
Epoch 46/3000
390/390 [==============================] - 37s 94ms/step - loss: 1.4750 - acc: 0.8109 - val_loss: 0.8575 - val_acc: 0.8329

Epoch 00046: val_acc improved from 0.83019 to 0.83288, saving model to model/mfcc6/LGD_fold3_co_resnet.h5
Epoch 47/3000
390/

390/390 [==============================] - 37s 94ms/step - loss: 1.3839 - acc: 0.8341 - val_loss: 0.8270 - val_acc: 0.8437

Epoch 00083: val_acc did not improve from 0.85445
Epoch 84/3000
390/390 [==============================] - 37s 94ms/step - loss: 1.3952 - acc: 0.8300 - val_loss: 0.8255 - val_acc: 0.8464

Epoch 00084: val_acc did not improve from 0.85445
Epoch 85/3000
390/390 [==============================] - 37s 94ms/step - loss: 1.4005 - acc: 0.8264 - val_loss: 0.8381 - val_acc: 0.8383

Epoch 00085: val_acc did not improve from 0.85445
Epoch 86/3000
390/390 [==============================] - 37s 94ms/step - loss: 1.3965 - acc: 0.8236 - val_loss: 0.8263 - val_acc: 0.8383

Epoch 00086: val_acc did not improve from 0.85445
Epoch 87/3000
390/390 [==============================] - 37s 94ms/step - loss: 1.4006 - acc: 0.8304 - val_loss: 0.8248 - val_acc: 0.8437

Epoch 00087: val_acc did not improve from 0.85445
Epoch 88/3000
390/390 [==============================] - 37s 94ms/step - l

Epoch 1/3000


ResourceExhaustedError: OOM when allocating tensor with shape[128,512,8,54] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_15/Adam/gradients/AddN_359-1-TransposeNHWCToNCHW-LayoutOptimizer = Transpose[T=DT_FLOAT, Tperm=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_15/Adam/gradients/batch_normalization_115/cond/FusedBatchNorm/Switch_grad/cond_grad, PermConstNHWCToNCHW-LayoutOptimizer)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: batch_normalization_143/Shape/_117841 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3353_batch_normalization_143/Shape", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: 

In [ ]:
#MFCC7
#0=>0.84367(resnet3_semi)/0.83288(not semi)
#1=>0.84367(resnet3_semi)/0.81132(not semi)
#2=>0.84097(resnet2_semi)/0.81671(not semi)
#3=>0.88410(resnet1_semi)/0.84367
#4=>0.82210(resnet4_semi)/0.74663(not semi)
#5 => 0.82749(resnet4_semi)/0.77628
#6 => 0.85445(resnet2_semi)/0.78437
#7 => 0.82749(resnet3_semi)/0.76280
#8 => 0.83288(resnet3_semi)/0.78706
# 9=> 0.90566(resnet1_semi)/0.87332

# 0=>0.88140 (co)
# 1 => 0.85714 (co)
# 2=>0.87062 (co)
# 3=>0.88949 (co)
# 4=>0.82749 (co)
# 5=>0.84367 (co)
# 6=>0.84906(X) (semi)
# 7=>0.84906 (co)
# 8=>0.83019(X) (smi)
# 9=>0.90566 (co)

In [ ]:
#MFCC6
# 0=> 0.81941 (not semi)
# 1=>0.83019 (semi)
# 2=>0.81941 (semi)
# 3=>0.85984 (resnet1_not semi)、0.78437
# 4=>0.84367 (renet1_not semi)、0.81132
# 5=> 0.85175(resnet3_semi)、0.82749(not semi)
#6 => 0.85904(resnet4_semi)、0.79784(not semi)
# 7=>0.88949(resnet2_semi)、0.83558(not semi)
# 8=>0.83558(resnet1 not semi)
# 9=>0.86792(resnet2_semi)、0.8112(not semi)

# 0 => 0.81941
#1=>0.88410
#2=>0.83558
# 3=>0.84367 (X)
# 4+>0.86523 (X)
# 5=> (X) (前五個fold街從頭train)

# 3=>0.85445 (x)
# 4=>0.82110 (x)
# 5=>0.84636 (X)
# 6=>0.86792
# 7=>0.88949
# 8=>0.88410
# 9=>0.87332